# Генеративные сети

Рекуррентные нейронные сети (RNN) и их варианты с управляемыми ячейками, такие как ячейки долгой краткосрочной памяти (LSTM) и управляемые рекуррентные блоки (GRU), предоставляют механизм для языкового моделирования, то есть они могут изучать порядок слов и предсказывать следующее слово в последовательности. Это позволяет использовать RNN для **генеративных задач**, таких как обычная генерация текста, машинный перевод и даже создание подписей к изображениям.

В архитектуре RNN, которую мы обсуждали в предыдущем разделе, каждая единица RNN выдавала следующее скрытое состояние в качестве результата. Однако мы также можем добавить еще один выход к каждой рекуррентной единице, что позволит нам выдавать **последовательность** (равную по длине исходной последовательности). Более того, мы можем использовать RNN-единицы, которые не принимают входные данные на каждом шаге, а просто используют некоторый начальный вектор состояния и затем генерируют последовательность выходных данных.

В этом блокноте мы сосредоточимся на простых генеративных моделях, которые помогают нам генерировать текст. Для упрощения давайте построим **символьную сеть**, которая генерирует текст по буквам. Во время обучения нам нужно взять некоторый текстовый корпус и разбить его на последовательности букв.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Создание словаря символов

Для построения генеративной сети на уровне символов необходимо разбить текст на отдельные символы, а не на слова. Это можно сделать, определив другой токенизатор:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Давайте посмотрим пример того, как мы можем закодировать текст из нашего набора данных:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Обучение генеративной RNN

Метод, который мы будем использовать для обучения RNN генерации текста, следующий. На каждом шаге мы берем последовательность символов длиной `nchars` и просим сеть сгенерировать следующий выходной символ для каждого входного символа:

![Изображение, показывающее пример генерации слова 'HELLO' с помощью RNN.](../../../../../translated_images/ru/rnn-generate.56c54afb52f9781d.webp)

В зависимости от конкретного сценария, мы также можем включить специальные символы, такие как *конец последовательности* `<eos>`. В нашем случае мы хотим обучить сеть бесконечной генерации текста, поэтому фиксируем размер каждой последовательности равным `nchars` токенов. Таким образом, каждый обучающий пример будет состоять из `nchars` входов и `nchars` выходов (входная последовательность, сдвинутая на один символ влево). Минипакет будет состоять из нескольких таких последовательностей.

Метод генерации минипакетов заключается в том, чтобы взять каждый текст новости длиной `l` и создать из него все возможные комбинации входов и выходов (таких комбинаций будет `l-nchars`). Они будут формировать один минипакет, и размер минипакетов будет различаться на каждом этапе обучения.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Теперь давайте определим генераторную сеть. Она может быть основана на любой рекуррентной ячейке, которую мы обсуждали в предыдущем разделе (простая, LSTM или GRU). В нашем примере мы будем использовать LSTM.

Поскольку сеть принимает символы в качестве входных данных, а размер словаря довольно небольшой, нам не нужен слой эмбеддинга — вход, закодированный в формате one-hot, может напрямую поступать в ячейку LSTM. Однако, так как мы передаем номера символов в качестве входных данных, их необходимо закодировать в формате one-hot перед передачей в LSTM. Это выполняется вызовом функции `one_hot` во время прохода `forward`. Кодировщик выхода будет представлен линейным слоем, который преобразует скрытое состояние в выход, закодированный в формате one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Во время обучения мы хотим иметь возможность выбирать сгенерированный текст. Для этого мы определим функцию `generate`, которая будет создавать выходную строку длиной `size`, начиная с начальной строки `start`.

Как это работает: сначала мы пропускаем всю начальную строку через сеть, получаем выходное состояние `s` и следующий предсказанный символ `out`. Поскольку `out` закодирован в формате one-hot, мы используем `argmax`, чтобы получить индекс символа `nc` в словаре, а затем с помощью `itos` определяем фактический символ и добавляем его в результирующий список символов `chars`. Этот процесс генерации одного символа повторяется `size` раз, чтобы сгенерировать необходимое количество символов.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Теперь приступим к обучению! Цикл обучения практически такой же, как и во всех наших предыдущих примерах, но вместо точности мы выводим сгенерированный текст каждые 1000 эпох.

Особое внимание следует уделить способу вычисления функции потерь. Нам нужно вычислить функцию потерь, используя одноразрядный закодированный вывод `out` и ожидаемый текст `text_out`, который представляет собой список индексов символов. К счастью, функция `cross_entropy` ожидает ненормализованный вывод сети в качестве первого аргумента и номер класса в качестве второго, что как раз соответствует нашим данным. Она также автоматически выполняет усреднение по размеру минипакета.

Мы также ограничиваем обучение количеством образцов, заданным в `samples_to_train`, чтобы не ждать слишком долго. Мы рекомендуем вам экспериментировать и попробовать более длительное обучение, возможно, на протяжении нескольких эпох (в этом случае вам нужно будет создать дополнительный цикл вокруг этого кода).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Этот пример уже генерирует довольно хороший текст, но его можно улучшить несколькими способами:
* **Улучшенная генерация минибатчей**. Способ, которым мы подготовили данные для обучения, заключался в создании одного минибатча из одного образца. Это не идеально, потому что минибатчи получаются разного размера, и некоторые из них вообще невозможно сгенерировать, если текст меньше, чем `nchars`. Кроме того, маленькие минибатчи недостаточно загружают GPU. Было бы разумнее взять один большой фрагмент текста из всех образцов, затем сгенерировать все пары вход-выход, перемешать их и создать минибатчи одинакового размера.
* **Многослойный LSTM**. Имеет смысл попробовать 2 или 3 слоя LSTM-ячейк. Как мы упоминали в предыдущем разделе, каждый слой LSTM извлекает определенные шаблоны из текста, и в случае генератора на уровне символов можно ожидать, что нижний уровень LSTM будет отвечать за извлечение слогов, а более высокие уровни — за слова и их комбинации. Это можно легко реализовать, передав параметр количества слоев в конструктор LSTM.
* Вы также можете поэкспериментировать с **GRU-ячейками** и посмотреть, какие из них работают лучше, а также с **разными размерами скрытых слоев**. Слишком большой скрытый слой может привести к переобучению (например, сеть будет запоминать точный текст), а слишком маленький размер может не дать хорошего результата.


## Генерация мягкого текста и температура

В предыдущем определении функции `generate` мы всегда выбирали символ с наибольшей вероятностью в качестве следующего символа в генерируемом тексте. Это приводило к тому, что текст часто "зацикливался" на одних и тех же последовательностях символов снова и снова, как в этом примере:
```
today of the second the company and a second the company ...
```

Однако, если мы посмотрим на распределение вероятностей для следующего символа, может оказаться, что разница между несколькими наивысшими вероятностями незначительна, например, один символ может иметь вероятность 0.2, а другой — 0.19 и т.д. Например, при поиске следующего символа в последовательности '*play*', следующим символом с равной вероятностью может быть пробел или **e** (как в слове *player*).

Это приводит нас к выводу, что не всегда "справедливо" выбирать символ с наивысшей вероятностью, так как выбор второго по величине вероятности также может привести к осмысленному тексту. Более разумным подходом будет **выбор** символов из распределения вероятностей, предоставленного выходом сети.

Этот выбор можно осуществить с помощью функции `multinomial`, которая реализует так называемое **мультиномиальное распределение**. Функция, реализующая эту **мягкую** генерацию текста, определена ниже:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Мы ввели еще один параметр, называемый **temperature**, который используется для указания того, насколько строго мы должны придерживаться наивысшей вероятности. Если temperature равен 1.0, мы выполняем честное мультиномиальное выборочное моделирование, а когда temperature стремится к бесконечности, все вероятности становятся равными, и мы случайным образом выбираем следующий символ. В приведенном ниже примере можно заметить, что текст становится бессмысленным, когда мы слишком сильно увеличиваем temperature, и он напоминает "циклический" жестко сгенерированный текст, когда значение приближается к 0.



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
